# Reading Data from external sources
<img src="https://user-images.githubusercontent.com/7065401/68501079-0695df00-023c-11ea-841f-455dac84a089.jpg"
    style="width:400px; float: right; margin: 0 40px 40px 40px;"></img>
We have seen that pandas is powerful enough to read data from multiple data source like csv, db, etc. Here we will see in more depth how we can use pandas to read and write data from multiple sources.

Rather than creating `Series` or `DataFrames` strutures from scratch, or even from Python core sequences or `ndarrays`, the most typical use of **pandas** is based on the loading of information from files or sources of information for further exploration, transformation and analysis.

In [1]:
import pandas as pd

## Reading CSV and TXT files

In this lecture we'll learn how to read comma-separated values files (.csv) and raw text files (.txt) into pandas `DataFrame`s.

### Reading data with Python

As we saw on previous courses we can read data simply using Python.

When you want to work with a file, the first thing to do is to open it. This is done by invoking the `open()` built-in function.

`open()` has a single required argument that is the path to the file and has a single return, the file object.

The `with` statement automatically takes care of closing the file once it leaves the `with` block, even in cases of error.

In [2]:
with open('data/btc-market-price-csv-read.csv', 'r') as fp:
    print(fp)

<_io.TextIOWrapper name='data/btc-market-price-csv-read.csv' mode='r' encoding='UTF-8'>


Once the file is opened, we can read its content as follows:

In [3]:
with open('data/btc-market-price-csv-read.csv', 'r') as fp:
    for index, line in enumerate(fp.readlines()):
        # read just the first 10 lines
        if (index < 10):
            print(index, line)

0 2/4/17 0:00,1099.169125

1 3/4/17 0:00,1141.813

2 4/4/17 0:00,?

3 5/4/17 0:00,1133.079314

4 6/4/17 0:00,-

5 7/4/17 0:00,-

6 8/4/17 0:00,1181.149838

7 9/4/17 0:00,1208.8005

8 10/4/17 0:00,1207.744875

9 11/4/17 0:00,1226.617038



How can we process the data read from the file using pure Python? It involves a lot of manual work, for example, splitting the values by the correct separator:

In [4]:
with open('data/btc-market-price-csv-read.csv', 'r') as fp:
    for index, line in enumerate(fp.readlines()):
        # read just the first 10 lines
        if (index < 10):
            timestamp, price = line.split(',')
            print(f"{timestamp}: ${price}")

2/4/17 0:00: $1099.169125

3/4/17 0:00: $1141.813

4/4/17 0:00: $?

5/4/17 0:00: $1133.079314

6/4/17 0:00: $-

7/4/17 0:00: $-

8/4/17 0:00: $1181.149838

9/4/17 0:00: $1208.8005

10/4/17 0:00: $1207.744875

11/4/17 0:00: $1226.617038



But what happens if the separator is unknown, like in the file `exam_review.csv`:

In [5]:
!head data/exam_review.csv

first_name>last_name>age>math_score>french_score
Ray>Morley>18>"68,000">"75,000"
Melvin>Scott>24>77>83
Amirah>Haley>22>92>67

Gerard>Mills>19>"78,000">72
Amy>Grimes>23>91>81


In this case, the separator is not a _comma_, but the `>` sign. It's still a "CSV", although not technically separated by commas.

#### The `csv` module

Python includes the builtin module `csv` that helps a little bit more with the process of reading CSVs:

In [6]:
import csv

In [7]:
with open('data/btc-market-price-csv-read.csv', 'r') as fp:
    reader = csv.reader(fp)
    for index, (timestamp, price) in enumerate(reader):
        # read just the first 10 lines
        if (index < 10):
            print(f"{timestamp}: ${price}")

2/4/17 0:00: $1099.169125
3/4/17 0:00: $1141.813
4/4/17 0:00: $?
5/4/17 0:00: $1133.079314
6/4/17 0:00: $-
7/4/17 0:00: $-
8/4/17 0:00: $1181.149838
9/4/17 0:00: $1208.8005
10/4/17 0:00: $1207.744875
11/4/17 0:00: $1226.617038


The `csv` modules takes care of splitting the file using a given separator (called `delimiter`) and creating an iterator for us.

In [8]:
with open('data/exam_review.csv', 'r') as fp:
    reader = csv.reader(fp, delimiter='>')  # special delimiter
    next(reader)  # skipping header
    for index, values in enumerate(reader):
        if not values:
            continue  # skip empty lines
        fname, lname, age, math, french = values
        print(f"{fname} {lname} (age {age}) got {math} in Math and {french} in French")

Ray Morley (age 18) got 68,000 in Math and 75,000 in French
Melvin Scott (age 24) got 77 in Math and 83 in French
Amirah Haley (age 22) got 92 in Math and 67 in French
Gerard Mills (age 19) got 78,000 in Math and 72 in French
Amy Grimes (age 23) got 91 in Math and 81 in French


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Reading data with Pandas

Probably one of the most recurrent types of work for data analysis: public data sources, logs, historical information tables, exports from databases. So the pandas library offers us functions to read and write files in multiple formats like CSV, JSON, XML and Excel's XLSX, all of them creating a `DataFrame` with the information read from the file.

We'll learn how to read different type of data including:
- CSV files (.csv)
- Raw text files (.txt)
- JSON data from a file and from an API
- Data from a SQL query over a database

There are many other available reading functions as the following table shows:

![pandas read data table](https://user-images.githubusercontent.com/7065401/68400151-51d5c200-0156-11ea-9732-aa00400c8e4b.png)

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### The `read_csv` method

The first method we'll learn is **read_csv**, that let us read comma-separated values (CSV) files and raw text (TXT) files into a `DataFrame`.

The `read_csv` function is extremely powerful and you can specify a very broad set of parameters at import time that allow us to accurately configure how the data will be read and parsed by specifying the correct structure, enconding and other details. The most common parameters are as follows:

- `filepath`: Path of the file to be read.
- `sep`: Character(s) that are used as a field separator in the file.
- `header`: Index of the row containing the names of the columns (None if none).
- `index_col`: Index of the column or sequence of indexes that should be used as index of rows of the data.
- `names`: Sequence containing the names of the columns (used together with header = None).
- `skiprows`: Number of rows or sequence of row indexes to ignore in the load.
- `na_values`: Sequence of values that, if found in the file, should be treated as NaN.
- `dtype`: Dictionary in which the keys will be column names and the values will be types of NumPy to which their content must be converted.
- `parse_dates`: Flag that indicates if Python should try to parse data with a format similar to dates as dates. You can enter a list of column names that must be joined for the parsing as a date.
- `date_parser`: Function to use to try to parse dates.
- `nrows`: Number of rows to read from the beginning of the file.
- `skip_footer`: Number of rows to ignore at the end of the file.
- `encoding`: Encoding to be expected from the file read.
- `squeeze`: Flag that indicates that if the data read only contains one column the result is a Series instead of a DataFrame.
- `thousands`: Character to use to detect the thousands separator.
- `decimal`: Character to use to detect the decimal separator.
- `skip_blank_lines`: Flag that indicates whether blank lines should be ignored.

> Full `read_csv` documentation can be found here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html.

In this case we'll try to read our `btc-market-price.csv` CSV file using different parameters to parse it correctly.

This file contains records of the mean price of Bitcoin per date.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Reading our first CSV file

Everytime we call `read_csv` method, we'll need to pass an explicit `filepath` parameter indicating the path where our CSV file is.

Any valid string path is acceptable. The string could be a URL. Valid URL schemes include HTTP, FTP, S3, and file. For file URLs, a host is expected. A local file could be: `file://localhost/path/to/table.csv`.

For example we can use `read_csv` method to load data directly from an URL:

In [9]:
csv_url = "https://raw.githubusercontent.com/datasets/gdp/master/data/gdp.csv"

pd.read_csv(csv_url).head()

,Country Name,Country Code,Year,Value
0,Arab World,ARB,1968,2.576068e+10
1,Arab World,ARB,1969,2.843420e+10
2,Arab World,ARB,1970,3.138550e+10
3,Arab World,ARB,1971,3.642691e+10
4,Arab World,ARB,1972,4.331606e+10


Or just use a local file:

In [10]:
df = pd.read_csv('data/btc-market-price-csv-read.csv')

df.head()

,2/4/17 0:00,1099.169125
0,3/4/17 0:00,1141.813
1,4/4/17 0:00,?
2,5/4/17 0:00,1133.079314
3,6/4/17 0:00,-
4,7/4/17 0:00,-


In this case we let pandas infer everything related to our data, but in most of the cases we'll need to explicitly tell pandas how we want our data to be loaded. To do that we use parameters.

Let's see how theses parameters work.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### First row behaviour with `header` parameter

The CSV file we're reading has only two columns: `Timestamp` and `Price`. It doesn't have a header. Pandas automatically assigned the first row of data as headers, which is incorrect. We can overwrite this behavior with the `header` parameter.

In [11]:
df = pd.read_csv('data/btc-market-price-csv-read.csv',
                 header=None)

In [12]:
df.head()

,0,1
0,2/4/17 0:00,1099.169125
1,3/4/17 0:00,1141.813
2,4/4/17 0:00,?
3,5/4/17 0:00,1133.079314
4,6/4/17 0:00,-


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Missing values with `na_values` parameter

We can define a `na_values` parameter with the values we want to be recognized as NA/NaN. In this case empty strings `''`, `?` and `-` will be recognized as null values.

In [13]:
df = pd.read_csv('data/btc-market-price-csv-read.csv',
                 header=None,
                 na_values=['', '?', '-'])

In [14]:
df.head()

,0,1
0,2/4/17 0:00,1099.169125
1,3/4/17 0:00,1141.813000
2,4/4/17 0:00,NaN
3,5/4/17 0:00,1133.079314
4,6/4/17 0:00,NaN


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Column names using `names` parameter

We'll add that columns names using the `names` parameter. 

In [15]:
df = pd.read_csv('data/btc-market-price-csv-read.csv',
                 header=None,
                 na_values=['', '?', '-'],
                 names=['Timestamp', 'Price'])

In [16]:
df.head()

,Timestamp,Price
0,2/4/17 0:00,1099.169125
1,3/4/17 0:00,1141.813000
2,4/4/17 0:00,NaN
3,5/4/17 0:00,1133.079314
4,6/4/17 0:00,NaN


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Timestamp  365 non-null    object 
 1   Price      354 non-null    float64
dtypes: float64(1), object(1)
memory usage: 5.8+ KB


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Column types using `dtype` parameter


Without using the `dtype` parameter pandas will try to figure it out the type of each column automatically. We can use `dtype` parameter to force pandas to use certain dtype.

In this case we'll force the `Price` column to be `float`.

In [18]:
df = pd.read_csv('data/btc-market-price-csv-read.csv',
                 header=None,
                 na_values=['', '?', '-'],
                 names=['Timestamp', 'Price'],
                 dtype={'Price': 'float'})

In [19]:
df.head()

,Timestamp,Price
0,2/4/17 0:00,1099.169125
1,3/4/17 0:00,1141.813000
2,4/4/17 0:00,NaN
3,5/4/17 0:00,1133.079314
4,6/4/17 0:00,NaN


In [20]:
df.dtypes

Timestamp     object
Price        float64
dtype: object

The `Timestamp` column was interpreted as a regular string (`object` in pandas notation), we can parse it manually using a vectorized operation as we saw on previous courses.

We'll parse `Timestamp` column to `Datetime` objects using `to_datetime` method:

In [21]:
pd.to_datetime(df['Timestamp']).head()

0   2017-02-04
1   2017-03-04
2   2017-04-04
3   2017-05-04
4   2017-06-04
Name: Timestamp, dtype: datetime64[ns]

In [22]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [23]:
df.head()

,Timestamp,Price
0,2017-02-04,1099.169125
1,2017-03-04,1141.813000
2,2017-04-04,NaN
3,2017-05-04,1133.079314
4,2017-06-04,NaN


In [24]:
df.dtypes

Timestamp    datetime64[ns]
Price               float64
dtype: object

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Date parser using `parse_dates` parameter

Another way of dealing with `Datetime` objects is using `parse_dates` parameter with the position of the columns with dates.

In [25]:
df = pd.read_csv('data/btc-market-price-csv-read.csv',
                 header=None,
                 na_values=['', '?', '-'],
                 names=['Timestamp', 'Price'],
                 dtype={'Price': 'float'},
                 parse_dates=[0])

In [26]:
df.head()

,Timestamp,Price
0,2017-02-04,1099.169125
1,2017-03-04,1141.813000
2,2017-04-04,NaN
3,2017-05-04,1133.079314
4,2017-06-04,NaN


In [27]:
df.dtypes

Timestamp    datetime64[ns]
Price               float64
dtype: object

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Adding index to our data using `index_col` parameter

By default, pandas will automatically assign a numeric autoincremental index or row label starting with zero. You may want to leave the default index as such if your data doesn’t have a column with unique values that can serve as a better index. In case there is a column that you feel would serve as a better index, you can override the default behavior by setting `index_col` property to a column. It takes a numeric value representing the index or a string of the column name for setting a single column as index or a list of numeric values or strings for creating a multi-index.

In our data, we are choosing the first column, `Timestamp`, as index (index=0) by passing zero to the `index_col` argument.

In [28]:
df = pd.read_csv('data/btc-market-price-csv-read.csv',
                 header=None,
                 na_values=['', '?', '-'],
                 names=['Timestamp', 'Price'],
                 dtype={'Price': 'float'},
                 parse_dates=[0],
                 index_col=[0])

In [29]:
df.head()

,Price
Timestamp,
2017-02-04,1099.169125
2017-03-04,1141.813000
2017-04-04,NaN
2017-05-04,1133.079314
2017-06-04,NaN


In [30]:
df.dtypes

Price    float64
dtype: object

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### A more challenging parsing

Now we'll read another CSV file. This file has the following columns:

- `first_name`
- `last_name`
- `age`
- `math_score`
- `french_score`
- `next_test_date`

Let's read it and see how it looks like.

In [31]:
exam_df = pd.read_csv('data/exam_review.csv')

In [32]:
exam_df

,,first_name>last_name>age>math_score>french_score
"Ray>Morley>18>""68","000"">""75","000"""
Melvin>Scott>24>77>83,NaN,NaN
Amirah>Haley>22>92>67,NaN,NaN
"Gerard>Mills>19>""78","000"">72",NaN
Amy>Grimes>23>91>81,NaN,NaN


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Custom data delimiters using `sep` parameter

We can define which delimiter to use by using the `sep` parameter. If we don't use the `sep` parameter, pandas will automatically detect the separator.

In most of the CSV files separator will be comma (`,`) and will be automatically detected. But we can find files with other separators like semicolon (`;`), tabs (`\t`, specially on TSV files), whitespaces or any other special character.

In this case the separator is a `>` character.

In [33]:
exam_df = pd.read_csv('data/exam_review.csv',
                      sep='>')

In [34]:
exam_df

,first_name,last_name,age,math_score,french_score
0,Ray,Morley,18,"68,000","75,000"
1,Melvin,Scott,24,77,83
2,Amirah,Haley,22,92,67
3,Gerard,Mills,19,"78,000",72
4,Amy,Grimes,23,91,81


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Custom data encoding

Files are stored using different "encodings". You've probably heard about ASCII, UTF-8, latin1, etc.

While reading data custom encoding can be defined with the `encoding` parameter.

- `encoding='UTF-8'`: will be used if data is UTF-8 encoded.
- `encoding='iso-8859-1'`: will be used if data is ISO/IEC 8859-1 ("extended ASCII") encoded.

In our case we don't need a custom enconding as data is properly loaded.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Custom numeric `decimal` and `thousands` character  

The decimal and thousands characters could change between datasets. If we have a column containing a comma (`,`) to indicate the decimal or thousands place, then this column would be considered a string and not numeric.

In [35]:
exam_df = pd.read_csv('data/exam_review.csv',
                      sep='>')

In [36]:
exam_df

,first_name,last_name,age,math_score,french_score
0,Ray,Morley,18,"68,000","75,000"
1,Melvin,Scott,24,77,83
2,Amirah,Haley,22,92,67
3,Gerard,Mills,19,"78,000",72
4,Amy,Grimes,23,91,81


In [37]:
exam_df[['math_score', 'french_score']].dtypes

math_score      object
french_score    object
dtype: object

To solve that, ensuring such columns are interpreted as integer values, we'll need to use the `decimal` and/or `thousands` parameters to indicate correct decimal and/or thousands indicators.

In [38]:
exam_df = pd.read_csv('data/exam_review.csv',
                      sep='>',
                      decimal=',')

In [39]:
exam_df

,first_name,last_name,age,math_score,french_score
0,Ray,Morley,18,68.0,75.0
1,Melvin,Scott,24,77.0,83.0
2,Amirah,Haley,22,92.0,67.0
3,Gerard,Mills,19,78.0,72.0
4,Amy,Grimes,23,91.0,81.0


In [40]:
exam_df[['math_score', 'french_score']].dtypes

math_score      float64
french_score    float64
dtype: object

Let's see what happens with the `thousands` parameter:

In [41]:
pd.read_csv('data/exam_review.csv',
            sep='>',
            thousands=',')

,first_name,last_name,age,math_score,french_score
0,Ray,Morley,18,68000,75000
1,Melvin,Scott,24,77,83
2,Amirah,Haley,22,92,67
3,Gerard,Mills,19,78000,72
4,Amy,Grimes,23,91,81


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Excluding specific rows

We can use the `skiprows` to:

- Exclude reading specified number of rows from the beginning of a file, by passing an integer argument. **This removes the header too**.
- Skip reading specific row indices from a file, by passing a list containing row indices to skip.

In [42]:
exam_df = pd.read_csv('data/exam_review.csv',
                      sep='>',
                      decimal=',')

In [43]:
exam_df

,first_name,last_name,age,math_score,french_score
0,Ray,Morley,18,68.0,75.0
1,Melvin,Scott,24,77.0,83.0
2,Amirah,Haley,22,92.0,67.0
3,Gerard,Mills,19,78.0,72.0
4,Amy,Grimes,23,91.0,81.0


To skip reading the first 2 rows from this file, we can use `skiprows=2`:

In [44]:
pd.read_csv('data/exam_review.csv',
            sep='>',
            skiprows=2)

,Melvin,Scott,24,77,83
0,Amirah,Haley,22,92,67
1,Gerard,Mills,19,"78,000",72
2,Amy,Grimes,23,91,81


As the header is considered as the first row, to skip reading data rows 1 and 3, we can use `skiprows=[1,3]`:

In [45]:
exam_df = pd.read_csv('data/exam_review.csv',
                      sep='>',
                      decimal=',',
                      skiprows=[1,3])

In [46]:
exam_df

,first_name,last_name,age,math_score,french_score
0,Melvin,Scott,24,77.0,83
1,Gerard,Mills,19,78.0,72
2,Amy,Grimes,23,91.0,81


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Get rid of blank lines

The `skip_blank_lines` parameter is set to `True` so blank lines are skipped while we read files.

If we set this parameter to `False`, then every blank line will be loaded with `NaN` values into the `DataFrame`.

In [47]:
pd.read_csv('data/exam_review.csv',
            sep='>',
            skip_blank_lines=False)

,first_name,last_name,age,math_score,french_score
0,Ray,Morley,18.0,"68,000","75,000"
1,Melvin,Scott,24.0,77,83
2,Amirah,Haley,22.0,92,67
3,NaN,NaN,NaN,NaN,NaN
4,Gerard,Mills,19.0,"78,000",72
5,Amy,Grimes,23.0,91,81


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Loading specific columns

We can use the `usecols` parameter when we want to load just specific columns and not all of them.

Performance wise, it is better because instead of loading an entire dataframe into memory and then deleting the not required columns, we can select the columns that we’ll need, while loading the dataset itself.

As a parameter to `usecols`, you can pass either a list of strings corresponding to the column names or a list of integers corresponding to column index.

In [48]:
pd.read_csv('data/exam_review.csv',
            usecols=['first_name', 'last_name', 'age'],
            sep='>')

,first_name,last_name,age
0,Ray,Morley,18
1,Melvin,Scott,24
2,Amirah,Haley,22
3,Gerard,Mills,19
4,Amy,Grimes,23


Or using just the column position:

In [49]:
pd.read_csv('data/exam_review.csv',
            usecols=[0, 1, 2],
            sep='>')

,first_name,last_name,age
0,Ray,Morley,18
1,Melvin,Scott,24
2,Amirah,Haley,22
3,Gerard,Mills,19
4,Amy,Grimes,23


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Using a `Series` instead of `DataFrame`

If the parsed data only contains one column then we can return a Series by setting the `squeeze` parameter to `True`.

In [50]:
exam_test_1 = pd.read_csv('data/exam_review.csv',
                          sep='>',
                          usecols=['last_name'])

In [51]:
exam_test_1

,last_name
0,Morley
1,Scott
2,Haley
3,Mills
4,Grimes


In [52]:
type(exam_test_1)

pandas.core.frame.DataFrame

In [53]:
exam_test_2 = pd.read_csv('data/exam_review.csv',
                          sep='>',
                          usecols=['last_name'],
                          squeeze=True)

In [54]:
exam_test_2

0    Morley
1     Scott
2     Haley
3     Mills
4    Grimes
Name: last_name, dtype: object

In [55]:
type(exam_test_2)

pandas.core.series.Series

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Save to CSV file

Finally we can also save our `DataFrame` as a CSV file.

In [56]:
exam_df

,first_name,last_name,age,math_score,french_score
0,Melvin,Scott,24,77.0,83
1,Gerard,Mills,19,78.0,72
2,Amy,Grimes,23,91.0,81


We can simply generate a CSV string from our `DataFrame`:

In [57]:
exam_df.to_csv()

',first_name,last_name,age,math_score,french_score\n0,Melvin,Scott,24,77.0,83\n1,Gerard,Mills,19,78.0,72\n2,Amy,Grimes,23,91.0,81\n'

Or specify a file path where we want our generated CSV code to be saved:

In [58]:
exam_df.to_csv('data/out-csv.csv')

In [59]:
pd.read_csv('data/out-csv.csv')

,Unnamed: 0,first_name,last_name,age,math_score,french_score
0,0,Melvin,Scott,24,77.0,83
1,1,Gerard,Mills,19,78.0,72
2,2,Amy,Grimes,23,91.0,81


In [60]:
exam_df.to_csv('data/out-csv.csv',
               index=None)

In [61]:
pd.read_csv('data/out-csv.csv')

,first_name,last_name,age,math_score,french_score
0,Melvin,Scott,24,77.0,83
1,Gerard,Mills,19,78.0,72
2,Amy,Grimes,23,91.0,81


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## Reading data from relational databases

In this lesson you will learn how to read SQL queries and relational database tables into `DataFrame` objects using pandas. Also, we'll take a look at different techniques to persist that pandas `DataFrame` objects to database tables.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Read data from SQL database

Reading data from SQL relational databases is fairly simple and pandas support a variety of methods to deal with it.

We'll start with an example using SQLite, as it's a builtin Python package, and we don't need anything extra installed.

In [62]:
import sqlite3

In order to work with a SQLite database from Python, we first have to connect to it. We can do that using the connect function, which returns a `Connection` object.

We'll use the following database structure:

<center>
<img src="./Images/chinook-dbstructure.png" width="700px" align="center" />
</center>

In [63]:
conn = sqlite3.connect('data/chinook.db')

Once we have a `Connection` object, we can then create a `Cursor` object. Cursors allow us to execute SQL queries against a database:

In [64]:
cur = conn.cursor()

The `Cursor` created has a method `execute`, which will receive SQL parameters to run against the database.

The code below will fetch the first `5` rows from the `employees` table:

In [65]:
cur.execute('SELECT * FROM employees LIMIT 5;')

You may have noticed that we didn't assign the result of the above query to a variable. This is because we need to run another command to actually fetch the results.

We can use the `fetchall` method to fetch all of the results of a query:

In [66]:
results = cur.fetchall()

In [67]:
results

[(1,
  'Adams',
  'Andrew',
  'General Manager',
  None,
  '1962-02-18 00:00:00',
  '2002-08-14 00:00:00',
  '11120 Jasper Ave NW',
  'Edmonton',
  'AB',
  'Canada',
  'T5K 2N1',
  '+1 (780) 428-9482',
  '+1 (780) 428-3457',
  'andrew@chinookcorp.com'),
 (2,
  'Edwards',
  'Nancy',
  'Sales Manager',
  1,
  '1958-12-08 00:00:00',
  '2002-05-01 00:00:00',
  '825 8 Ave SW',
  'Calgary',
  'AB',
  'Canada',
  'T2P 2T3',
  '+1 (403) 262-3443',
  '+1 (403) 262-3322',
  'nancy@chinookcorp.com'),
 (3,
  'Peacock',
  'Jane',
  'Sales Support Agent',
  2,
  '1973-08-29 00:00:00',
  '2002-04-01 00:00:00',
  '1111 6 Ave SW',
  'Calgary',
  'AB',
  'Canada',
  'T2P 5M5',
  '+1 (403) 262-3443',
  '+1 (403) 262-6712',
  'jane@chinookcorp.com'),
 (4,
  'Park',
  'Margaret',
  'Sales Support Agent',
  2,
  '1947-09-19 00:00:00',
  '2003-05-03 00:00:00',
  '683 10 Street SW',
  'Calgary',
  'AB',
  'Canada',
  'T2P 5G3',
  '+1 (403) 263-4423',
  '+1 (403) 263-4289',
  'margaret@chinookcorp.com'),
 (5,


As you can see, the results are returned as a list of tuples. Each tuple corresponds to a row in the database that we accessed. Dealing with data this way is painful.

We'd need to manually add column headers, and manually parse the data. Luckily, the pandas library has an easier way, which we'll look at in the next section.

In [68]:
df = pd.DataFrame(results)

In [69]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


Before we move on, it's good practice to close `Connection` objects and `Cursor` objects that are open. This prevents the SQLite database from being locked. When a SQLite database is locked, you may be unable to update the database, and may get errors. We can close the Cursor and the Connection like this:

In [70]:
cur.close()
conn.close()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Using pandas `read_sql` method

We can use the pandas `read_sql` function to read the results of a SQL query directly into a pandas `DataFrame`. The code below will execute the same query that we just did, but it will return a `DataFrame`. It has several advantages over the query we did above:

- It doesn't require us to create a `Cursor` object or call `fetchall` at the end.
- It automatically reads in the names of the headers from the table.
- It creates a `DataFrame`, so we can quickly explore the data.

In [71]:
conn = sqlite3.connect('data/chinook.db')

In [72]:
df = pd.read_sql('SELECT * FROM employees;', conn)

In [73]:
df.head()

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


In [74]:
df = pd.read_sql('SELECT * FROM employees;', conn,
                 index_col='EmployeeId',
                 parse_dates=['BirthDate', 'HireDate'])

In [75]:
df.head()

,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
EmployeeId,,,,,,,,,,,,,,
1,Adams,Andrew,General Manager,NaN,1962-02-18,2002-08-14,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1.0,1958-12-08,2002-05-01,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29,2002-04-01,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2.0,1947-09-19,2003-05-03,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03,2003-10-17,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 1 to 8
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   LastName    8 non-null      object        
 1   FirstName   8 non-null      object        
 2   Title       8 non-null      object        
 3   ReportsTo   7 non-null      float64       
 4   BirthDate   8 non-null      datetime64[ns]
 5   HireDate    8 non-null      datetime64[ns]
 6   Address     8 non-null      object        
 7   City        8 non-null      object        
 8   State       8 non-null      object        
 9   Country     8 non-null      object        
 10  PostalCode  8 non-null      object        
 11  Phone       8 non-null      object        
 12  Fax         8 non-null      object        
 13  Email       8 non-null      object        
dtypes: datetime64[ns](2), float64(1), object(11)
memory usage: 960.0+ bytes


In [77]:
df['ReportsTo'].isna().sum()

1

In [78]:
df['ReportsTo'].mean()

2.857142857142857

In [79]:
df['ReportsTo'] > 1.75

EmployeeId
1    False
2    False
3     True
4     True
5     True
6    False
7     True
8     True
Name: ReportsTo, dtype: bool

In [80]:
df['City'] = df['City'].astype('category')

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 1 to 8
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   LastName    8 non-null      object        
 1   FirstName   8 non-null      object        
 2   Title       8 non-null      object        
 3   ReportsTo   7 non-null      float64       
 4   BirthDate   8 non-null      datetime64[ns]
 5   HireDate    8 non-null      datetime64[ns]
 6   Address     8 non-null      object        
 7   City        8 non-null      category      
 8   State       8 non-null      object        
 9   Country     8 non-null      object        
 10  PostalCode  8 non-null      object        
 11  Phone       8 non-null      object        
 12  Fax         8 non-null      object        
 13  Email       8 non-null      object        
dtypes: category(1), datetime64[ns](2), float64(1), object(10)
memory usage: 1008.0+ bytes


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Using pandas `read_sql_query` method

It turns out that the `read_sql` method we saw above is just a wrapper around `read_sql_query` and `read_sql_table`.

We can get the same result using `read_sql_query` method:

In [82]:
conn = sqlite3.connect('data/chinook.db')

In [83]:
df = pd.read_sql_query('SELECT * FROM employees LIMIT 5;', conn)

In [84]:
df.head()

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


In [85]:
df = pd.read_sql_query('SELECT * FROM employees;', conn,
                       index_col='EmployeeId',
                       parse_dates=['BirthDate', 'HireDate'])

In [86]:
df.head()

,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
EmployeeId,,,,,,,,,,,,,,
1,Adams,Andrew,General Manager,NaN,1962-02-18,2002-08-14,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1.0,1958-12-08,2002-05-01,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29,2002-04-01,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2.0,1947-09-19,2003-05-03,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03,2003-10-17,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Using `read_sql_table` method

`read_sql_table` is a useful function, but it works only with [SQLAlchemy](https://www.sqlalchemy.org/), a Python SQL Toolkit and Object Relational Mapper.

This is just a demonstration of its usage where we read the whole `employees` table.

In [87]:
from sqlalchemy import create_engine

In [88]:
engine = create_engine('sqlite:///data/chinook.db')

connection = engine.connect()

In [89]:
df = pd.read_sql_table('employees', con=connection)

In [90]:
df.head()

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18,2002-08-14,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08,2002-05-01,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29,2002-04-01,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19,2003-05-03,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03,2003-10-17,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


In [91]:
df = pd.read_sql_table('employees', con=connection,
                       index_col='EmployeeId',
                       parse_dates=['BirthDate', 'HireDate'])

In [92]:
df.head()

,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
EmployeeId,,,,,,,,,,,,,,
1,Adams,Andrew,General Manager,NaN,1962-02-18,2002-08-14,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1.0,1958-12-08,2002-05-01,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29,2002-04-01,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2.0,1947-09-19,2003-05-03,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03,2003-10-17,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


In [93]:
connection.close()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Create tables from `DataFrame` objects

Finally we can persist `DataFrame` objects we've working on in a database using the pandas `to_sql` method.

Although it is easy to implement, it could be a very slow process.

In [94]:
df.head()

,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
EmployeeId,,,,,,,,,,,,,,
1,Adams,Andrew,General Manager,NaN,1962-02-18,2002-08-14,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1.0,1958-12-08,2002-05-01,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29,2002-04-01,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2.0,1947-09-19,2003-05-03,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03,2003-10-17,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


_Drop the table if needed_

In [95]:
df.to_sql?

Signature:
df.to_sql(
    name: str,
    con,
    schema=None,
    if_exists: str = 'fail',
    index: bool = True,
    index_label=None,
    chunksize=None,
    dtype=None,
    method=None,
) -> None
Docstring:
Write records stored in a DataFrame to a SQL database.

Databases supported by SQLAlchemy [1]_ are supported. Tables can be
newly created, appended to, or overwritten.

Parameters
----------
name : str
    Name of SQL table.
con : sqlalchemy.engine.Engine or sqlite3.Connection
    Using SQLAlchemy makes it possible to use any DB supported by that
    library. Legacy support is provided for sqlite3.Connection objects. The user
    is responsible for engine disposal and connection closure for the SQLAlchemy
    connectable See `here                 <https://docs.sqlalchemy.org/en/13/core/connections.html>`_

schema : str, optional
    Specify the schema (if database flavor supports this). If None, use
    default schema.
if_exists : {'fail', 'replace', 'append'}, default 'fail'
 

In [96]:
cur = conn.cursor()

In [97]:
cur.execute('DROP TABLE IF EXISTS employees2;')

In [98]:
cur.close()

In [99]:
df.to_sql('employees2', conn)

In [100]:
pd.read_sql_query('SELECT * FROM employees2;', conn).head()

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Custom behavior

The `if_exists` parameter define how to behave if the table already exists and adds a ton of flexibility, letting you decide wheather to `replace` current table data, `append` new data at the end, or simply `fail` if table already exists.

In [101]:
pd.DataFrame().to_sql('employees2',
                      conn,
                      if_exists='replace')

In [102]:
pd.read_sql_query('SELECT * FROM employees2;', conn).head()

,index


In [103]:
df.to_sql('employees2',
          conn,
          if_exists='replace')

In [104]:
pd.read_sql_query('SELECT * FROM employees2;', conn).head()

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


In [105]:
conn.close()

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## Reading Excel files

In this lecture we'll learn how to read Excel files (.xlsx) and its sheets into a pandas `DataFrame`s, and how to export that `DataFrame`s to different sheets and Excel files using the pandas `ExcelWriter` and `to_excel` methods.

In [106]:
!head data/products.xlsx

�<M�2�Հb�)���p�6 N����k��nK|q�g���X�o�d9��� +�Z$��&�� ���7�ɗ`��dK0�5a���;��B��j�^�yZ��.�~! �	�(  �  xl/_rels/workbook.xml.rels �(�                                                                                                                                                                                                                                                                 ���j�0���}q�ne�:��A�[� &Q���6��'o?�C�@�.��$�}?Ч����j�U�%���Z�(�8�><� ֶҝ��`@�Cq��Ns�D��$��%��襤�`�)qm��.��c�uy�                                                          ��MO�0��H�����ݐBKwAH�!T~�I����$ݿ'T�G�~����<���!��4��;#�w����qu*&r�Fq���v�����GJy(v��*����K��#F��D��.W	��=��Z�MY�b���BS�����7��ϛז��
������O	/�c)�wڇ��b�  �� PK    ! ���5  �     xl/workbook.xml�U]O�0}����=8N��F�R�!�	1>���"�3ۡE���k�)-��A�����9��:G˺���B6"{�x�d!��]]��y�Ц��lx��FG�߿,���Iy�@�3TӦkV��=��F�R��@S�a�*N]rn�
�A�����R�9�Ƨ�u5o�
���8��S��C���C���=2�L��������%Cr`%R.�

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### The `read_excel` method

We'll begin with the **read_excel** method, that let us read Excel files into a `DataFrame`.

This method supports both XLS and XLSX file extensions from a local filesystem or URL and has a broad set of parameters to configure how the data will be read and parsed. These parameters are very similar to the parameters we saw on previous lectures where we introduced the `read_csv` method. The most common parameters are as follows:

- `filepath`: Path of the file to be read.
- `sheet_name`: Strings are used for sheet names. Integers are used in zero-indexed sheet positions. Lists of strings/integers are used to request multiple sheets. Specify None to get all sheets.
- `header`: Index of the row containing the names of the columns (None if none).
- `index_col`: Index of the column or sequence of indexes that should be used as index of rows of the data.
- `names`: Sequence containing the names of the columns (used together with header = None).
- `skiprows`: Number of rows or sequence of row indexes to ignore in the load.
- `na_values`: Sequence of values that, if found in the file, should be treated as NaN.
- `dtype`: Dictionary in which the keys will be column names and the values will be types of NumPy to which their content must be converted.
- `parse_dates`: Flag that indicates if Python should try to parse data with a format similar to dates as dates. You can enter a list of column names that must be joined for the parsing as a date.
- `date_parser`: Function to use to try to parse dates.
- `nrows`: Number of rows to read from the beginning of the file.
- `skip_footer`: Number of rows to ignore at the end of the file.
- `squeeze`: Flag that indicates that if the data read only contains one column the result is a Series instead of a DataFrame.
- `thousands`: Character to use to detect the thousands separator.

> Full `read_excel` documentation can be found here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html.

In this case we'll try to read our `products.xlsx` Excel file.

This file contains records of products with its price, brand, description and merchant information on different sheets.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Reading our first Excel file

Everytime we call `read_excel` method, we'll need to pass an explicit `filepath` parameter indicating the path where our Excel file is.

Any valid string path is acceptable. The string could be a URL. Valid URL schemes include HTTP, FTP, S3, and file. For file URLs, a host is expected. A local file could be: `file://localhost/path/to/table.xlsx`.

In [107]:
df = pd.read_excel('data/products.xlsx')

In [108]:
df.head()

,product_id,price,merchant_id,brand,name
0,AVphzgbJLJeJML43fA0o,104.99,1001,Sanus,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...
1,AVpgMuGwLJeJML43KY_c,69.00,1002,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...
2,AVpe9FXeLJeJML43zHrq,23.99,1001,DENAQ,DENAQ - AC Adapter for TOSHIBA SATELLITE
3,AVpfVJXu1cnluZ0-iwTT,290.99,1001,DreamWave,DreamWave - Tremor Portable Bluetooth Speaker ...
4,AVphUeKeilAPnD_x3-Be,244.01,1004,Yamaha,NS-SP1800BL 5.1-Channel Home Theater System (B...


In this case we let pandas infer everything related to our data, but in most of the cases we'll need to explicitly tell pandas how we want our data to be loaded. To do that we use parameters.

Let's see how theses parameters work.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### First row behaviour with `header` parameter

The Excel file we're reading has the following columns:

- `product_id`
- `price`
- `merchant_id`
- `brand`
- `name`

The first row (0-index) of the data has that column names, so we keep the implicit `header=0` parameter to let Pandas assign this first row as headers. We can overwrite this behavior defining explicitly the `header` parameter.

In [109]:
pd.read_excel('data/products.xlsx').head()

,product_id,price,merchant_id,brand,name
0,AVphzgbJLJeJML43fA0o,104.99,1001,Sanus,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...
1,AVpgMuGwLJeJML43KY_c,69.00,1002,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...
2,AVpe9FXeLJeJML43zHrq,23.99,1001,DENAQ,DENAQ - AC Adapter for TOSHIBA SATELLITE
3,AVpfVJXu1cnluZ0-iwTT,290.99,1001,DreamWave,DreamWave - Tremor Portable Bluetooth Speaker ...
4,AVphUeKeilAPnD_x3-Be,244.01,1004,Yamaha,NS-SP1800BL 5.1-Channel Home Theater System (B...


In [110]:
pd.read_excel('data/products.xlsx',
              header=None).head()

,0,1,2,3,4
0,product_id,price,merchant_id,brand,name
1,AVphzgbJLJeJML43fA0o,104.99,1001,Sanus,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...
2,AVpgMuGwLJeJML43KY_c,69,1002,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...
3,AVpe9FXeLJeJML43zHrq,23.99,1001,DENAQ,DENAQ - AC Adapter for TOSHIBA SATELLITE
4,AVpfVJXu1cnluZ0-iwTT,290.99,1001,DreamWave,DreamWave - Tremor Portable Bluetooth Speaker ...


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Adding index to our data using `index_col` parameter

By default, pandas will automatically assign a numeric autoincremental index or row label starting with zero.

You may want to leave the default index as such if your data doesn’t have a column with unique values that can serve as a better index.

In case there is a column that you feel would serve as a better index, you can override the default behavior by setting `index_col` property to a column. It takes a numeric value or a string for setting a single column as index or a list of numeric values for creating a multi-index.

In our data, we are choosing the first column, `product_id`, as index (index=0) by passing zero to the `index_col` argument.

In [111]:
df = pd.read_excel('data/products.xlsx',
                   index_col=[0])

In [112]:
df.head()

,price,merchant_id,brand,name
product_id,,,,
AVphzgbJLJeJML43fA0o,104.99,1001,Sanus,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...
AVpgMuGwLJeJML43KY_c,69.00,1002,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...
AVpe9FXeLJeJML43zHrq,23.99,1001,DENAQ,DENAQ - AC Adapter for TOSHIBA SATELLITE
AVpfVJXu1cnluZ0-iwTT,290.99,1001,DreamWave,DreamWave - Tremor Portable Bluetooth Speaker ...
AVphUeKeilAPnD_x3-Be,244.01,1004,Yamaha,NS-SP1800BL 5.1-Channel Home Theater System (B...


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Selecting specific sheets

Excel files quite often have multiple sheets and the ability to read a specific sheet or all of them is very important. To make this easy, the pandas `read_excel` method takes an argument called `sheet_name` that tells pandas which sheet to read in the data from.

For this, you can either use the sheet name or the sheet number. Sheet numbers start with zero. The first sheet will be the one loaded by default. You can change sheet by specifying `sheet_name` parameter.

In [113]:
products = pd.read_excel('data/products.xlsx',
                         sheet_name='Products',
                         index_col='product_id')

In [114]:
products.head()

,price,merchant_id,brand,name
product_id,,,,
AVphzgbJLJeJML43fA0o,104.99,1001,Sanus,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...
AVpgMuGwLJeJML43KY_c,69.00,1002,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...
AVpe9FXeLJeJML43zHrq,23.99,1001,DENAQ,DENAQ - AC Adapter for TOSHIBA SATELLITE
AVpfVJXu1cnluZ0-iwTT,290.99,1001,DreamWave,DreamWave - Tremor Portable Bluetooth Speaker ...
AVphUeKeilAPnD_x3-Be,244.01,1004,Yamaha,NS-SP1800BL 5.1-Channel Home Theater System (B...


In [115]:
merchants = pd.read_excel('data/products.xlsx',
                          sheet_name='Merchants',
                          index_col='merchant_id')

In [116]:
merchants.head()

,merchant
merchant_id,
1001,Bestbuy.com
1002,Walmart.com
1003,Bestbuy.com
1004,Growkart
1005,bhphotovideo.com


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### The `ExcelFile` class

Another approach on reading Excel data is using the `ExcelFile` class for parsing tabular Excel sheets into `DataFrame` objects.

This `ExcelFile` will let us work with sheets easily, and will be faster than the previous `read_excel` method.

In [117]:
excel_file = pd.ExcelFile('data/products.xlsx')

In [118]:
excel_file

We can now explore the sheets on that Excel file with `sheet_names`:

In [119]:
excel_file.sheet_names

['Products', 'Descriptions', 'Merchants']

And parse specified sheet(s) into a Pandas' `DataFrame` using ExcelFile's `parse()` method.

Everytime we call `parse()` method, we'll need to pass an explicit `sheet_name` parameter indicating which sheet from the Excel file we want to be parsed. First sheet will be parsed by default.

In [120]:
products = excel_file.parse('Products')

In [121]:
products.head()

,product_id,price,merchant_id,brand,name
0,AVphzgbJLJeJML43fA0o,104.99,1001,Sanus,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...
1,AVpgMuGwLJeJML43KY_c,69.00,1002,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...
2,AVpe9FXeLJeJML43zHrq,23.99,1001,DENAQ,DENAQ - AC Adapter for TOSHIBA SATELLITE
3,AVpfVJXu1cnluZ0-iwTT,290.99,1001,DreamWave,DreamWave - Tremor Portable Bluetooth Speaker ...
4,AVphUeKeilAPnD_x3-Be,244.01,1004,Yamaha,NS-SP1800BL 5.1-Channel Home Theater System (B...


This `parse()` method has all the parameters we saw before on `read_excel()` method, let's try some of them:

In [122]:
products = excel_file.parse(sheet_name='Products',
                            header=0,
                            index_col='product_id')

In [123]:
products.head()

,price,merchant_id,brand,name
product_id,,,,
AVphzgbJLJeJML43fA0o,104.99,1001,Sanus,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...
AVpgMuGwLJeJML43KY_c,69.00,1002,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...
AVpe9FXeLJeJML43zHrq,23.99,1001,DENAQ,DENAQ - AC Adapter for TOSHIBA SATELLITE
AVpfVJXu1cnluZ0-iwTT,290.99,1001,DreamWave,DreamWave - Tremor Portable Bluetooth Speaker ...
AVphUeKeilAPnD_x3-Be,244.01,1004,Yamaha,NS-SP1800BL 5.1-Channel Home Theater System (B...


In [124]:
products.dtypes

price          float64
merchant_id      int64
brand           object
name            object
dtype: object

In [125]:
merchants = excel_file.parse('Merchants',
                             index_col='merchant_id')

In [126]:
merchants.head()

,merchant
merchant_id,
1001,Bestbuy.com
1002,Walmart.com
1003,Bestbuy.com
1004,Growkart
1005,bhphotovideo.com


In [127]:
merchants.dtypes

merchant    object
dtype: object

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Save to Excel file

Finally we can save our `DataFrame` as a Excel file.

In [128]:
products.head()

,price,merchant_id,brand,name
product_id,,,,
AVphzgbJLJeJML43fA0o,104.99,1001,Sanus,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...
AVpgMuGwLJeJML43KY_c,69.00,1002,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...
AVpe9FXeLJeJML43zHrq,23.99,1001,DENAQ,DENAQ - AC Adapter for TOSHIBA SATELLITE
AVpfVJXu1cnluZ0-iwTT,290.99,1001,DreamWave,DreamWave - Tremor Portable Bluetooth Speaker ...
AVphUeKeilAPnD_x3-Be,244.01,1004,Yamaha,NS-SP1800BL 5.1-Channel Home Theater System (B...


A fast, simple way to write a single `DataFrame` to an Excel file is to use the `to_excel()` method of the `DataFrame` directly.

Note that it's required to pass a output file path.

> The `OpenPyXL - openpyxl` library should be installed in order to save Excel files. `pip install openpyxl`

In [129]:
products.to_excel('data/out.xlsx')

In [130]:
pd.read_excel('data/out.xlsx').head()

,product_id,price,merchant_id,brand,name
0,AVphzgbJLJeJML43fA0o,104.99,1001,Sanus,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...
1,AVpgMuGwLJeJML43KY_c,69.00,1002,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...
2,AVpe9FXeLJeJML43zHrq,23.99,1001,DENAQ,DENAQ - AC Adapter for TOSHIBA SATELLITE
3,AVpfVJXu1cnluZ0-iwTT,290.99,1001,DreamWave,DreamWave - Tremor Portable Bluetooth Speaker ...
4,AVphUeKeilAPnD_x3-Be,244.01,1004,Yamaha,NS-SP1800BL 5.1-Channel Home Theater System (B...


We can specify the sheet name with `sheet_name` parameter:

In [131]:
products.to_excel('data/out.xlsx',
                  sheet_name='Products')

Further calls of `to_excel` with different sheet names will only overwrite the first sheet instead of adding additional sheets.

Also, be aware that by removing the index, we'll lose that column.

In [132]:
products.to_excel('data/out.xlsx',
                  index=None)

In [133]:
pd.read_excel('data/out.xlsx').head()

,price,merchant_id,brand,name
0,104.99,1001,Sanus,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...
1,69.00,1002,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...
2,23.99,1001,DENAQ,DENAQ - AC Adapter for TOSHIBA SATELLITE
3,290.99,1001,DreamWave,DreamWave - Tremor Portable Bluetooth Speaker ...
4,244.01,1004,Yamaha,NS-SP1800BL 5.1-Channel Home Theater System (B...


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Positioning Data with `startrow` and `startcol`

Suppose we wanted to insert the our data into the spreadsheet file in a position somewhere other than the top-left corner.

We can shift where the `to_excel` method writes the data by using `startrow` to set the cell after which the first row will be printed, and `startcol` to set which cell after which the first column will be printed.

<img src="https://user-images.githubusercontent.com/7065401/68594890-41378b80-0477-11ea-9ae4-ff87e5e1128d.png" />

In [134]:
products.to_excel('data/out.xlsx',
                  sheet_name='Products',
                  startrow=1,
                  startcol=2)

<img src="https://user-images.githubusercontent.com/7065401/68594828-249b5380-0477-11ea-87d7-af694c09f2d2.png" />

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Saving multiple sheets

If we wanted to write a single `DataFrame` to a single sheet with default formatting then we are done. However, if we want to write multiple sheets and/or multiple `DataFrame`s, then we will need to create an `ExcelWriter` object.

The `ExcelWriter` object is included in the Pandas module and is used to open Excel files and handle write operations. This object behaves almost exactly like the vanilla Python `open` object that we used on previous courses and can be used within a `with` block.

> When the `ExcelWriter` object is executed, any existing file with the same name as the output file will be overwritten.

In [135]:
writer = pd.ExcelWriter('data/out.xlsx')

In [136]:
writer

Instead of including the file pathname in the `to_excel` call, we will use the `ExcelWriter` object `writer` instead.

In [137]:
with writer:
    products.to_excel(writer, sheet_name='Products')

In [138]:
pd.read_excel('data/out.xlsx', sheet_name='Products').head()

,product_id,price,merchant_id,brand,name
0,AVphzgbJLJeJML43fA0o,104.99,1001,Sanus,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...
1,AVpgMuGwLJeJML43KY_c,69.00,1002,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...
2,AVpe9FXeLJeJML43zHrq,23.99,1001,DENAQ,DENAQ - AC Adapter for TOSHIBA SATELLITE
3,AVpfVJXu1cnluZ0-iwTT,290.99,1001,DreamWave,DreamWave - Tremor Portable Bluetooth Speaker ...
4,AVphUeKeilAPnD_x3-Be,244.01,1004,Yamaha,NS-SP1800BL 5.1-Channel Home Theater System (B...


We can now add another `Merchants` sheet simply using the `writer` object:

In [139]:
with writer:
    merchants.to_excel(writer, sheet_name='Merchants')

In [140]:
pd.read_excel('data/out.xlsx', sheet_name='Products').head()

,product_id,price,merchant_id,brand,name
0,AVphzgbJLJeJML43fA0o,104.99,1001,Sanus,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...
1,AVpgMuGwLJeJML43KY_c,69.00,1002,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...
2,AVpe9FXeLJeJML43zHrq,23.99,1001,DENAQ,DENAQ - AC Adapter for TOSHIBA SATELLITE
3,AVpfVJXu1cnluZ0-iwTT,290.99,1001,DreamWave,DreamWave - Tremor Portable Bluetooth Speaker ...
4,AVphUeKeilAPnD_x3-Be,244.01,1004,Yamaha,NS-SP1800BL 5.1-Channel Home Theater System (B...


In [141]:
pd.read_excel('data/out.xlsx', sheet_name='Merchants').head()

,merchant_id,merchant
0,1001,Bestbuy.com
1,1002,Walmart.com
2,1003,Bestbuy.com
3,1004,Growkart
4,1005,bhphotovideo.com


Or we can save multiple sheets at the same time:

In [142]:
with pd.ExcelWriter('data/out.xlsx') as writer:
    products.to_excel(writer, sheet_name='Products')
    merchants.to_excel(writer, sheet_name='Merchants')

In that case the resulting `out.xlxs` file will have two sheets `Products` and `Merchants`.

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## Reading HTML tables

In this lecture we'll learn how to read and parse HTML tables from websites into a list of `DataFrame` objects to work with.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Parsing raw HTML strings

Another useful pandas method is `read_html()`. This method will read HTML tables from a given URL, a file-like object, or a raw string containing HTML, and return a list of `DataFrame` objects.

Let's try to read the following `html_string` into a `DataFrame`.

_(Open data/sample.html for the working example)_

In [143]:
html_string = """
<table>
    <thead>
      <tr>
        <th>Order date</th>
        <th>Region</th> 
        <th>Item</th>
        <th>Units</th>
        <th>Unit cost</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>1/6/2018</td>
        <td>East</td> 
        <td>Pencil</td>
        <td>95</td>
        <td>1.99</td>
      </tr>
      <tr>
        <td>1/23/2018</td>
        <td>Central</td> 
        <td>Binder</td>
        <td>50</td>
        <td>19.99</td>
      </tr>
      <tr>
        <td>2/9/2018</td>
        <td>Central</td> 
        <td>Pencil</td>
        <td>36</td>
        <td>4.99</td>
      </tr>
      <tr>
        <td>3/15/2018</td>
        <td>West</td> 
        <td>Pen</td>
        <td>27</td>
        <td>19.99</td>
      </tr>
    </tbody>
</table>
"""

In [144]:
from IPython.core.display import display, HTML
display(HTML(html_string))

Order date,Region,Item,Units,Unit cost
1/6/2018,East,Pencil,95,1.99
1/23/2018,Central,Binder,50,19.99
2/9/2018,Central,Pencil,36,4.99
3/15/2018,West,Pen,27,19.99


In [145]:
dfs = pd.read_html(html_string)

The `read_html` just returned one `DataFrame` object:

In [146]:
len(dfs)

1

In [147]:
df = dfs[0]

df

,Order date,Region,Item,Units,Unit cost
0,1/6/2018,East,Pencil,95,1.99
1,1/23/2018,Central,Binder,50,19.99
2,2/9/2018,Central,Pencil,36,4.99
3,3/15/2018,West,Pen,27,19.99


Previous `DataFrame` looks quite similar to the raw HTML table, but now we have a `DataFrame` object, so we can apply any pandas operation we want to it.

In [148]:
df.shape

(4, 5)

In [149]:
df.loc[df['Region'] == 'Central']

,Order date,Region,Item,Units,Unit cost
1,1/23/2018,Central,Binder,50,19.99
2,2/9/2018,Central,Pencil,36,4.99


In [150]:
df.loc[df['Units'] > 35]

,Order date,Region,Item,Units,Unit cost
0,1/6/2018,East,Pencil,95,1.99
1,1/23/2018,Central,Binder,50,19.99
2,2/9/2018,Central,Pencil,36,4.99


#### Defining header

Pandas will automatically find the header to use thanks to the <thead> tag.
    
But in many cases we'll find wrong or incomplete tables that make the `read_html` method parse the tables in a wrong way without the proper headers.

To fix them we can use the `header` parameter.

In [151]:
html_string = """
<table>
  <tr>
    <td>Order date</td>
    <td>Region</td> 
    <td>Item</td>
    <td>Units</td>
    <td>Unit cost</td>
  </tr>
  <tr>
    <td>1/6/2018</td>
    <td>East</td> 
    <td>Pencil</td>
    <td>95</td>
    <td>1.99</td>
  </tr>
  <tr>
    <td>1/23/2018</td>
    <td>Central</td> 
    <td>Binder</td>
    <td>50</td>
    <td>19.99</td>
  </tr>
  <tr>
    <td>2/9/2018</td>
    <td>Central</td> 
    <td>Pencil</td>
    <td>36</td>
    <td>4.99</td>
  </tr>
  <tr>
    <td>3/15/2018</td>
    <td>West</td> 
    <td>Pen</td>
    <td>27</td>
    <td>19.99</td>
  </tr>
</table>
"""

In [152]:
pd.read_html(html_string)[0]

,0,1,2,3,4
0,Order date,Region,Item,Units,Unit cost
1,1/6/2018,East,Pencil,95,1.99
2,1/23/2018,Central,Binder,50,19.99
3,2/9/2018,Central,Pencil,36,4.99
4,3/15/2018,West,Pen,27,19.99


In this case, we'll need to pass the row number to use as header using the `header` parameter.

In [153]:
pd.read_html(html_string, header=0)[0]

,Order date,Region,Item,Units,Unit cost
0,1/6/2018,East,Pencil,95,1.99
1,1/23/2018,Central,Binder,50,19.99
2,2/9/2018,Central,Pencil,36,4.99
3,3/15/2018,West,Pen,27,19.99


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Parsing HTML tables from the web

Now that we know how `read_html` works, go one step beyond and try to parse HTML tables directly from an URL.

To do that we'll call the `read_html` method with an URL as paramter.

#### Simple example

In [154]:
html_url = "https://www.basketball-reference.com/leagues/NBA_2019_per_game.html"

In [155]:
nba_tables = pd.read_html(html_url)

In [156]:
len(nba_tables)

1

We'll work with the only one table found:

In [157]:
nba = nba_tables[0]

In [158]:
nba.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,...,.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,...,.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,3,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,...,.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,4,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,...,.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,5,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,...,.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9


In [159]:
nba.head(25)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,...,.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,...,.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,3,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,...,.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,4,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,...,.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,5,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,...,.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
5,6,Deng Adel,SF,21,CLE,19,3,10.2,0.6,1.9,...,1.000,0.2,0.8,1.0,0.3,0.1,0.2,0.3,0.7,1.7
6,7,DeVaughn Akoon-Purcell,SG,25,DEN,7,0,3.1,0.4,1.4,...,.500,0.1,0.4,0.6,0.9,0.3,0.0,0.3,0.6,1.0
7,8,LaMarcus Aldridge,C,33,SAS,81,81,33.2,8.4,16.3,...,.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3
8,9,Rawle Alkins,SG,21,CHI,10,1,12.0,1.3,3.9,...,.667,1.1,1.5,2.6,1.3,0.1,0.0,0.8,0.7,3.7
9,10,Grayson Allen,SG,23,UTA,38,2,10.9,1.8,4.7,...,.750,0.1,0.5,0.6,0.7,0.2,0.2,0.9,1.2,5.6


#### Complex example

We can also use the `requests` module to get HTML code from an URL to parse it into `DataFrame` objects.

If we look at the given URL we can see multiple tables about The Simpsons TV show.

We want to keep the table with information about each season.

In [160]:
import requests

html_url = "https://en.wikipedia.org/wiki/The_Simpsons"

In [161]:
r = requests.get(html_url)

wiki_tables = pd.read_html(r.text, header=0)

In [162]:
len(wiki_tables)

27

In [163]:
simpsons = wiki_tables[1]

In [164]:
simpsons.head()

,Season,Season.1,No. ofepisodes,Originally aired,Originally aired.1,Originally aired.2,Viewership,Viewership.1,Viewership.2
0,Season,Season,No. ofepisodes,Season premiere,Season finale,Time Slot (ET),Avg. viewers(in millions),Most watched episode,Most watched episode
1,Season,Season,No. ofepisodes,Season premiere,Season finale,Time Slot (ET),Avg. viewers(in millions),Viewers(millions),Episode Title
2,1,1989–90,13,"December 17, 1989","May 13, 1990",Sunday 8:30 pm,27.8,33.5,"""Life on the Fast Lane"""
3,2,1990–91,22,"October 11, 1990","July 11, 1991",Thursday 8:00 pm,24.4,33.6,"""Bart Gets an F"""
4,3,1991–92,24,"September 19, 1991","August 27, 1992",Thursday 8:00 pm,21.8,25.5,"""Colonel Homer"""


Quick clean on the table: remove extra header rows and set `Season` as index.

In [165]:
simpsons.drop([0, 1], inplace=True)

In [166]:
simpsons.set_index('Season', inplace=True)

Which season has the lowest number of episodes?

In [167]:
simpsons['No. ofepisodes'].unique()

array(['13', '22', '24', '25', '23', '21', '20'], dtype=object)

In [168]:
simpsons = simpsons.loc[simpsons['No. ofepisodes'] != 'TBA']

In [169]:
min_season = simpsons['No. ofepisodes'].min()

min_season

'13'

In [170]:
simpsons.loc[simpsons['No. ofepisodes'] == min_season]

,Season.1,No. ofepisodes,Originally aired,Originally aired.1,Originally aired.2,Viewership,Viewership.1,Viewership.2
Season,,,,,,,,
1,1989–90,13,"December 17, 1989","May 13, 1990",Sunday 8:30 pm,27.8,33.5,"""Life on the Fast Lane"""


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Save to CSV file

Finally save the `DataFrame` to a CSV file as we saw on previous lectures.

In [171]:
simpsons.head()

,Season.1,No. ofepisodes,Originally aired,Originally aired.1,Originally aired.2,Viewership,Viewership.1,Viewership.2
Season,,,,,,,,
1,1989–90,13,"December 17, 1989","May 13, 1990",Sunday 8:30 pm,27.8,33.5,"""Life on the Fast Lane"""
2,1990–91,22,"October 11, 1990","July 11, 1991",Thursday 8:00 pm,24.4,33.6,"""Bart Gets an F"""
3,1991–92,24,"September 19, 1991","August 27, 1992",Thursday 8:00 pm,21.8,25.5,"""Colonel Homer"""
4,1992–93,22,"September 24, 1992","May 13, 1993",Thursday 8:00 pm,22.4,28.6,"""Lisa's First Word"""
5,1993–94,22,"September 30, 1993","May 19, 1994",Thursday 8:00 pm,18.9,24.0,"""Treehouse of Horror IV"""


In [172]:
simpsons.to_csv('out-html.csv')

In [173]:
pd.read_csv('out-html.csv', index_col='Season').head()

,Season.1,No. ofepisodes,Originally aired,Originally aired.1,Originally aired.2,Viewership,Viewership.1,Viewership.2
Season,,,,,,,,
1,1989–90,13,"December 17, 1989","May 13, 1990",Sunday 8:30 pm,27.8,33.5,"""Life on the Fast Lane"""
2,1990–91,22,"October 11, 1990","July 11, 1991",Thursday 8:00 pm,24.4,33.6,"""Bart Gets an F"""
3,1991–92,24,"September 19, 1991","August 27, 1992",Thursday 8:00 pm,21.8,25.5,"""Colonel Homer"""
4,1992–93,22,"September 24, 1992","May 13, 1993",Thursday 8:00 pm,22.4,28.6,"""Lisa's First Word"""
5,1993–94,22,"September 30, 1993","May 19, 1994",Thursday 8:00 pm,18.9,24.0,"""Treehouse of Horror IV"""


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)